# Hello PyTorch Training Loop

In [1]:
# Prereruisites 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

### Create a dumb dataset

Pairs of numbers and their sum 

In [7]:
# Make a simple dummy dataset
class SimpleNumbersSumDataset(Dataset):
    def __init__(self, data_range=(1, 16)):
        self.numbers = list(range(data_range[0], data_range[1]))

    def __getitem__(self, idx):
        num1 = float(self.numbers[idx // len(self.numbers)])  
        num2 = float(self.numbers[idx % len(self.numbers)])  
        return torch.tensor([num1, num2]), torch.tensor([num1 + num2])

    def __len__(self):
        return len(self.numbers) ** 2

Check out the dataset

In [8]:
data_set = SimpleNumbersSumDataset()
for i in range(8):
    print(data_set[i])

(tensor([1., 1.]), tensor([2.]))
(tensor([1., 2.]), tensor([3.]))
(tensor([1., 3.]), tensor([4.]))
(tensor([1., 4.]), tensor([5.]))
(tensor([1., 5.]), tensor([6.]))
(tensor([1., 6.]), tensor([7.]))
(tensor([1., 7.]), tensor([8.]))
(tensor([1., 8.]), tensor([9.]))


### Build a simple dumb model

In [9]:
class SimpleMLP(nn.Module):
    def __init__(self, input_size):
        super(SimpleMLP, self).__init__()
        self.hidden_layer = nn.Linear(input_size, 128)
        self.output_layer = nn.Linear(128, 1)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.hidden_layer(x)) 
        x = self.output_layer(x)
        return x
    

### Instantiate all components

In [10]:
data_set = SimpleNumbersSumDataset(data_range=(0, 100))
data_loader = DataLoader(data_set, batch_size=100, shuffle=True)
model = SimpleMLP(input_size=2)
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Create PyTorch Training Loop

In [12]:
for epoch in range(20):  # Loop over the dataset multiple times
    total_loss = 0.0
    for number_pairs, sums in data_loader:   # Iterate over batches of data
        predictions = model(number_pairs)    # Forward pass
        loss = loss_function(predictions, sums)  # Compute the loss
        loss.backward()  # Perform backpropagation
        optimizer.step() # Update the weights
        optimizer.zero_grad() # Zero the gradients

        total_loss += loss.item()  # Accumulate the loss

    # print the loss this epoch
    print(f'Epoch {epoch} Loss: {total_loss:.4f}')

Epoch 0 Loss: 1.2220
Epoch 1 Loss: 1.1016
Epoch 2 Loss: 0.9940
Epoch 3 Loss: 0.9016
Epoch 4 Loss: 0.8158
Epoch 5 Loss: 0.7418
Epoch 6 Loss: 0.6636
Epoch 7 Loss: 0.6054
Epoch 8 Loss: 0.5443
Epoch 9 Loss: 0.4925
Epoch 10 Loss: 0.4516
Epoch 11 Loss: 0.4156
Epoch 12 Loss: 0.3816
Epoch 13 Loss: 0.3570
Epoch 14 Loss: 0.3341
Epoch 15 Loss: 0.3152
Epoch 16 Loss: 0.3019
Epoch 17 Loss: 0.2837
Epoch 18 Loss: 0.2711
Epoch 19 Loss: 0.2590


### Test the model

In [13]:
# Test with 5 + 6
model(torch.tensor([[5.0, 6.0]]))

tensor([[11.0119]], grad_fn=<AddmmBackward0>)

In [14]:
# Test with 25 + 36
model(torch.tensor([[25.0, 36.0]]))

tensor([[61.0070]], grad_fn=<AddmmBackward0>)